## Portfolios and Portfolio Groups

Portfolios are the main containers for transactions and holdings, and come in multiple forms based on hierarchy and use case. The basic portfolio types are :
- Transaction Portfolios
- Reference Portfolios
- Derived Portfolios

These portfolios can then be grouped, where portfolio operations on the group are applied to an aggregated set of portfolios in the hierarchy.

This example will show how to :
1. Create Transactions Portfolios
2. Create Derived Portfolios
3. (add/get/delete portfolio properties and details)
4. Create portfolio Groups
5. Add portfolios to a Group
6. Add a group to a group (creation of subgroups)
7. Modify groups
8. Remove portfolios and subgroups from groups
9. Delete portfolios
10. Delete groups
11. Get all the commands applied to a group or portfolio
12. List all the groups and portfolios in a scope

Initialise our environment and connect to LUSID

In [122]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
from msrest.authentication import BasicTokenAuthentication

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID version : ', client.api_version)

LUSID Environment Initialised
LUSID version :  0.9.134


First we start by loading some default portfolios, we will put them in two different scopes to demonstrate the separation between scopes and the usage of different scopes within portfolio groups.

- Note: In order to demonstrate the deletion more clearly at then end of this example, run the portfolio creation more than once, and the derived portfolio creation 2-3 times as well, this will create a population of unnecessary portfolios that can be safely removed without affecting the portfolios of interest. It is not necessary to 

### Initial Setup

In [123]:
# Use pandas to read a csv containing the information needed to create some portfolios
df = pd.read_csv("data/dummy_portfolios.csv")
print(df.head())

# We can also save the headers to a separate variable for easier use
print('')
headers = df.columns
print("number of columns : ", headers.size)
print("column names : ", headers)


                 scope    code      display_name base_currency
0  portfolio_demo_main  main_1  main_portfolio-1           GBP
1  portfolio_demo_main  main_2  main_portfolio-2           GBP
2  portfolio_demo_main  main_3  main_portfolio-3           GBP
3  portfolio_demo_main  main_4  main_portfolio-4           GBP
4  portfolio_demo_main  main_5  main_portfolio-5           GBP

number of columns :  4
column names :  Index(['scope', 'code', 'display_name', 'base_currency'], dtype='object')


In [124]:
# Create a transaction Portfolio - extract the scopes from the csv
distinct_scopes = df['scope'].unique()
scope1 = distinct_scopes[0]
scope2 = distinct_scopes[1]
print ("Scope1 : ", scope1)
print ("Scope2 : ", scope2)

Scope1 :  portfolio_demo_main
Scope2 :  portfolio_demo_second


In [132]:
# The following function creates a random alphanumeric code of 4 characters that can be appended to Ids 
# and Names to ensure they remain unique throughout multiple runs of this example
def GetGuid():
    return str(uuid.uuid4())[:4]

# The following function will create portfolios from our dataframe of csv data
def CreatePortfolioFromDataframe(df):
    effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)
    list_results = []
    for index, row in df.iterrows():
        print("Creating portfolio ", index)
        #print(row[headers[0]], row[headers[1]], row[headers[2]], row[headers[3]])
        guid = GetGuid()
        request = models.CreateTransactionPortfolioRequest(
            code = row[headers[1]]+guid,
            display_name = row[headers[2]]+guid,
            base_currency = row[headers[3]],
            created = effective_date,
            description = None, 
            corporate_action_source_id = None, 
            accounting_method = None, 
            sub_holding_keys = None, 
            properties = None)
        result = client.create_portfolio(
            scope = row[headers[0]], 
            create_request = request)
        list_results.append(result)
        print('...')
    return list_results

# The following function extracts resource ids from a list of portfolios
def GetResourceIdsFromResponse(responses):
    resource_id_list = []
    for portfolio in responses:
        resource_id_list.append(portfolio.id)
    return resource_id_list

# The following function prints the details obtained from 'GetPortfolioGroup'
def PrintGetPortfolioGroupResponse(response):
    print ('Name: ' + response.display_name)
    print ('Scope: ' + response.id.scope)
    print ('Code: ' + response.id.code)
    print ('Portfolios Inside Group: ')
    for portfolios in response.portfolios:
        print (portfolios.code)
    print ('Subgroups Inside Group: ')
    for subgroup in response.sub_groups:
        print (subgroup.code)
        
def portfolio_details_response(response):
    print(response.origin_portfolio_id.scope)
    print(response.origin_portfolio_id.code)
    print(response.base_currency)
    if hasattr(response, 'accounting_method'):
        print(response.accounting_method)
    if hasattr(response, 'corporate_action_source_id'):
        print(response.corporate_action_source_id)
    print('')

We begin by upserting the csv data into LUSID in the form of new transaction portfolios

In [133]:
created_portfolios = CreatePortfolioFromDataframe(df)

# we can now prettyprint all our portfolio responses to see the info on each new portfolio
for portfolio in created_portfolios :
    prettyprint.portfolio_response(portfolio)

Creating portfolio  0
...
Creating portfolio  1
...
Creating portfolio  2
...
Creating portfolio  3
...
Creating portfolio  4
...
Creating portfolio  5
...
Creating portfolio  6
...
Creating portfolio  7
...
Portfolio Created
Scope: portfolio_demo_main
Code: main_105e4
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:40.086152+00:00

Portfolio Created
Scope: portfolio_demo_main
Code: main_225a2
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:40.508499+00:00

Portfolio Created
Scope: portfolio_demo_main
Code: main_3ea4e
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:41.010701+00:00

Portfolio Created
Scope: portfolio_demo_main
Code: main_420cd
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:41.377553+00:00

Portfolio Created
Scope: portfolio_demo_main
Code: main_58ff3
Portfolio Effective From: 2018-01-01 00:00:00+00:0

We then create one derived portfolio from the first transaction portfolio created.

In [134]:
# Create derived transaction portfolio from the first portfolio in the list
guid = GetGuid()
derived_portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
        display_name = "derived_portfolio-{0}".format(guid),
        code = "derived_id-{0}".format(guid),
        parent_portfolio_id = models.ResourceId(
            scope = created_portfolios[0].id.scope,
            code = created_portfolios[0].id.code),
        description = "derived_portfolio_description",
        created = created_portfolios[0].created,
        corporate_action_source_id = None, 
        accounting_method = None, 
        sub_holding_keys = None)

derived_portfolio = client.create_derived_portfolio(
    scope = scope1,
    portfolio = derived_portfolio_request, 
    custom_headers = None, 
    raw = False)

prettyprint.portfolio_response(derived_portfolio)

Derived Portfolio Created
Scope: portfolio_demo_main
Code: derived_id-aec4
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:50.002992+00:00

Parent Portfolio Details
Code: main_105e4


A derived portfolio will automatically inherit all of the parent portfolio's details if they are not specified. These details can be modified and/or deleted. 
Modifying the details on a derived portfolio will not affect the parent portfolio. This is demonstrated in the following cells

In [135]:
# First we will get the details for our new derived portfolio 
# and check they match with the parent portfolio
derived_details = client.get_details(
    scope = scope1, 
    code = derived_portfolio.id.code, 
    effective_at = None, 
    as_at = None, 
    custom_headers = None, 
    raw = False)
print("Derived Portfolio Details : ")
portfolio_details_response(derived_details)

print("Parent Portfolio Details : ")
parent_details = client.get_details(
    scope = scope1, 
    code = created_portfolios[0].id.code, 
    effective_at = None, 
    as_at = None, 
    custom_headers = None, 
    raw = False)
portfolio_details_response(parent_details)

Derived Portfolio Details : 
portfolio_demo_main
main_105e4
GBP

Parent Portfolio Details : 
portfolio_demo_main
main_105e4
GBP



Now we can modify the derived portfolio currency so it no longer matches the parent portfolio

In [136]:
request = models.CreatePortfolioDetails(
    base_currency = "USD", 
    corporate_action_source_id = None)
response = client.upsert_portfolio_details(
    scope = scope1, 
    code = derived_portfolio.id.code, 
    details = request, 
    effective_at = None,
    custom_headers = None, 
    raw = False)
print("Derived Portfolio Details : ")
portfolio_details_response(response)

print("Parent Portfolio Details : ")
parent_details = client.get_details(
    scope = scope1, 
    code = created_portfolios[0].id.code, 
    effective_at = None, 
    as_at = None, 
    custom_headers = None, 
    raw = False)
portfolio_details_response(parent_details)

Derived Portfolio Details : 
portfolio_demo_main
derived_id-aec4
USD

Parent Portfolio Details : 
portfolio_demo_main
main_105e4
GBP



Now if we can delete portfolio details ---I DONT KNOW WHAT THIS DOES OTHER THAN BREAK GETDETAILS---

In [137]:
response = client.delete_derived_portfolio_details(
    scope = scope1, 
    code = derived_portfolio.id.code, 
    effective_at = None, 
    as_at = None, 
    custom_headers = None, 
    raw = False)
derived_details = client.get_details(
    scope = scope1, 
    code = derived_portfolio.id.code, 
    effective_at = None, 
    as_at = None, 
    custom_headers = None, 
    raw = False)
# print("Derived Portfolio Details : ")
# portfolio_details_response(derived_details)

print("Parent Portfolio Details : ")
parent_details = client.get_details(
    scope = scope1, 
    code = created_portfolios[0].id.code)
portfolio_details_response(parent_details)

Parent Portfolio Details : 
portfolio_demo_main
main_105e4
GBP



Next we can create some portfolio groups, add the portfolios to the groups, add one of the groups to the other to create a subgroup, and subsequently update its name to reflect this change in hierarchy.
Note that the second group is in a different scope to the first, it is entirely possible to combine portfolios and groups in different scopes, keep in mind that group-wide operations will still only apply to the specified scope, and will not affect portfolios/subgroups in different scopes.

In [138]:
# Create a portfolio group -- create two 
# -- The first group we will create with all the portfolios in the first scope 
# -- we can use a helper function to obtain a list of resource ids for this
scope_1_portfolios = [ (x) for x in created_portfolios if x.id.scope  == scope1 ]
scope_1_resource_ids = GetResourceIdsFromResponse(scope_1_portfolios)

# Create group 1 with the portfolios of scope 1 in it
guid = GetGuid()
group_request1 = models.CreatePortfolioGroupRequest(
    id = "portfolio_group-{0}".format(guid),
    display_name = "portfolio_group1_name",
    values = scope_1_resource_ids, 
    sub_groups = None, 
    description = None)

# Create group 2 with no portfolios
guid = GetGuid()
group_request2 = models.CreatePortfolioGroupRequest(
    id = "portfolio_group-{0}".format(guid),
    display_name = "portfolio_group2_name",
    values = None, 
    sub_groups = None, 
    description = None)

portfolio_group1 = client.create_portfolio_group(scope1, group_request1)
portfolio_group2 = client.create_portfolio_group(scope2, group_request2)

prettyprint.portfolio_group_response(portfolio_group1, 'created')
prettyprint.portfolio_group_response(portfolio_group2, 'created')



Portfolio Group Created
Name: portfolio_group1_name
Scope: portfolio_demo_main
Code: portfolio_group-a737
Portfolios Inside Group: 
main_105e4
main_225a2
main_3ea4e
main_420cd
main_58ff3
main_6bc81


Portfolio Group Created
Name: portfolio_group2_name
Scope: portfolio_demo_second
Code: portfolio_group-7dfb
Portfolios Inside Group: 




So we have one group pre-populated with the portfolios in the first scope `portfolio_demo_main` and we have one empty group in the second scope `portfolio_demo_second`. The derived portfolio hasn't been included in the first group, so we can add it in now to group 1, and we can add the remaining portfolios to group 2.

In [139]:
# Add portfolios to groups
# Once again we get a list of resource ids for the portfolios in scope 2 this time
scope_2_portfolios = [ (x) for x in created_portfolios if x.id.scope  == scope2 ]
scope_2_resource_ids = GetResourceIdsFromResponse(scope_2_portfolios)

# adding the derived portfolio to group 1 :
client.add_portfolio_to_group(
    scope = portfolio_group1.id.scope, 
    code = portfolio_group1.id.code, 
    portfolio_id = derived_portfolio.id,
    custom_headers = None, 
    raw = False)

# then to add multiple portfolios to group 2, we have to iterate over the list of ids, since they have to
# be added one by one.
for resource_id in scope_2_resource_ids:
    client.add_portfolio_to_group(
        scope = portfolio_group2.id.scope, 
        code = portfolio_group2.id.code, 
        portfolio_id = resource_id,
        custom_headers = None, 
        raw = False)

We can also combine groups into each other to create sub-groups. We will add group 2 as a subgroup in group 1.

In [140]:
# Add group to group
client.add_sub_group_to_group(
    scope = portfolio_group1.id.scope, 
    code = portfolio_group1.id.code, 
    portfolio_group_id = portfolio_group2.id,
    custom_headers = None, 
    raw = False)

Since group 2 is now a subgroup, we can update its display name to reflect this

In [141]:
# Update group request
group_update_request = models.UpdatePortfolioGroupRequest(
    display_name = "Group1_Subgroup", 
    description = None)

# Update group
client.update_portfolio_group(
    scope = portfolio_group2.id.scope, 
    code = portfolio_group2.id.code, 
    request = group_update_request,
    custom_headers = None, 
    raw = False)

We can now compare two methods of obtaining groups from LUSID. The first is `get portfolio group` while the second is `get portfolio group expansion`. The difference here is that the first method will return only information about the specified group, including the ids of the portfolios and subgroups it contains, but not any further details of its portfolios and subgroups. The second method will not only return this basic information, but also the full details of each portfolio in the group and each portfolio in every subgroup as well. 


First we compare group 1 at the time of creation and at the most up-to-date time below, so we can see the changes that have been made to it since we created it.

In [142]:
# Get portfolio group now
updated_group1 = client.get_portfolio_group(
    scope = portfolio_group1.id.scope, 
    code = portfolio_group1.id.code, 
    as_at = datetime.utcnow().replace(tzinfo = pytz.utc),
    custom_headers=None, 
    raw=False)

# Get portfolio group 1 prior to the update
original_group1 = client.get_portfolio_group(
    scope = portfolio_group1.id.scope, 
    code = portfolio_group1.id.code, 
    as_at = portfolio_group1.version.as_at_date,
    custom_headers=None, 
    raw=False)

print('Group 1 at time of creation')
PrintGetPortfolioGroupResponse(original_group1)
print('')
print('Group 1 at latest time')
PrintGetPortfolioGroupResponse(updated_group1)


Group 1 at time of creation
Name: portfolio_group1_name
Scope: portfolio_demo_main
Code: portfolio_group-a737
Portfolios Inside Group: 
main_105e4
main_225a2
main_3ea4e
main_420cd
main_58ff3
main_6bc81
Subgroups Inside Group: 

Group 1 at latest time
Name: portfolio_group1_name
Scope: portfolio_demo_main
Code: portfolio_group-a737
Portfolios Inside Group: 
main_105e4
main_225a2
main_3ea4e
main_420cd
main_58ff3
main_6bc81
derived_id-aec4
Subgroups Inside Group: 
portfolio_group-7dfb


We can now compare this method with getting the full expansion of a group. You will see that on top of a list of portfolios and subgroups, the full expansion also returns all the information for the portfolios similar to running the `get_portfolio` method on each portfolio in the group and groubgroup.

In [143]:
# Get full expansion of a portfolio group
expansion = client.get_portfolio_group_expansion(
    scope = portfolio_group1.id.scope, 
    code = portfolio_group1.id.code,
    effective_at=None, 
    as_at=None, 
    property_filter=None, 
    custom_headers=None, 
    raw=False)

prettyprint.expanded_portfolio_group_response(expansion)

Portfolio Group Full Details : 
Scope: portfolio_demo_main
Code: portfolio_group-a737
Portfolios Inside Group: 
Portfolio Created
Scope: portfolio_demo_main
Code: main_105e4
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:50.002992+00:00

Portfolio Created
Scope: portfolio_demo_main
Code: main_225a2
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:40.508499+00:00

Portfolio Created
Scope: portfolio_demo_main
Code: main_3ea4e
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:41.010701+00:00

Portfolio Created
Scope: portfolio_demo_main
Code: main_420cd
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:41.377553+00:00

Portfolio Created
Scope: portfolio_demo_main
Code: main_58ff3
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:42.449959+00:00

Portfolio Created
Scope: portfolio_dem

Suppose we wish to revert these changes, we have to explicitly delete each component step by step. Our Subgroup contains two portfolios, we're going to remove these from that group, remove the subgroup from the main group, and then delete it as it is no longer needed. If a subgroup is empty, it is not necessary to remove it before deleting it completely, however for example purposes we will follow each step.

In [144]:
#1. Removal of portfolios from a group has to be done one portfolio at a time, 
# thus we can loop through the list of portfolios in scope 2 since they are all contained in the subgroup
# first we print out the details of group 2
original_group2 = client.get_portfolio_group(
    scope = portfolio_group2.id.scope, 
    code = portfolio_group2.id.code,
    as_at = None,
    custom_headers=None, 
    raw=False)

PrintGetPortfolioGroupResponse(original_group2)
print('')

# then we proceed with removing each portfolio in the group, you will see each printed response will have
# one fewer portfolio ids attached to the group.
for portfolio_id in scope_2_resource_ids:
    portfolio_removal = client.delete_portfolio_from_group(
        scope = portfolio_group2.id.scope, 
        code = portfolio_group2.id.code, 
        portfolio_scope = portfolio_id.scope, 
        portfolio_code = portfolio_id.code,
        custom_headers=None, 
        raw=False)
    prettyprint.portfolio_group_response(portfolio_removal, 'none')


Name: Group1_Subgroup
Scope: portfolio_demo_second
Code: portfolio_group-7dfb
Portfolios Inside Group: 
second_1884c
second_2c207
Subgroups Inside Group: 

Portfolio Group
Name: Group1_Subgroup
Scope: portfolio_demo_second
Code: portfolio_group-7dfb
Portfolios Inside Group: 
second_2c207


Portfolio Group
Name: Group1_Subgroup
Scope: portfolio_demo_second
Code: portfolio_group-7dfb
Portfolios Inside Group: 




In [145]:
#2. Remove group from group
group_removal = client.delete_sub_group_from_group(
    scope = portfolio_group1.id.scope, 
    code = portfolio_group1.id.code, 
    subgroup_scope = portfolio_group2.id.scope, 
    subgroup_code = portfolio_group2.id.code,
    custom_headers=None, 
    raw=False)

prettyprint.portfolio_group_response(group_removal, 'none')
print('Remaining Sub Groups : ', group_removal.sub_groups)

Portfolio Group
Name: portfolio_group1_name
Scope: portfolio_demo_main
Code: portfolio_group-a737
Portfolios Inside Group: 
main_105e4
main_225a2
main_3ea4e
main_420cd
main_58ff3
main_6bc81
derived_id-aec4


Remaining Sub Groups :  []


In [146]:
#3. Delete group
group_deletion = client.delete_portfolio_group(
    scope = portfolio_group2.id.scope, 
    code = portfolio_group2.id.code,
    custom_headers=None, 
    raw=False)

We can now check what operations have been applied to each of our groups, including an already deleted group, by using the `get portfolio group commands` method, as below.

In [147]:
# Get commands
group1_commands = client.get_portfolio_group_commands(
    scope = portfolio_group1.id.scope, 
    code = portfolio_group1.id.code,
    from_as_at=None, 
    to_as_at=None, 
    sort_by=None, 
    start=None, 
    limit=None, 
    filter=None, 
    custom_headers=None, 
    raw=False)

group2_commands = client.get_portfolio_group_commands(
    scope = portfolio_group2.id.scope, 
    code = portfolio_group2.id.code,
    from_as_at=None, 
    to_as_at=None, 
    sort_by=None, 
    start=None, 
    limit=None, 
    filter=None, 
    custom_headers=None, 
    raw=False)

prettyprint.group_commands(group1_commands, portfolio_group1.display_name)

prettyprint.group_commands(group2_commands, portfolio_group2.display_name)

Commands Applied To Group portfolio_group1_name
Number of commands : 4
Description : Create portfolio group
At Time : 2019-02-18T13:28:02.0170640+00:00


Description : Add portfolio to group
At Time : 2019-02-18T13:28:04.6277960+00:00


Description : Add subgroup to group
At Time : 2019-02-18T13:28:08.9544530+00:00


Description : Delete subgroup from group
At Time : 2019-02-18T13:30:46.6209270+00:00


Commands Applied To Group portfolio_group2_name
Number of commands : 7
Description : Create portfolio group
At Time : 2019-02-18T13:28:02.3815850+00:00


Description : Add portfolio to group
At Time : 2019-02-18T13:28:05.0480870+00:00


Description : Add portfolio to group
At Time : 2019-02-18T13:28:05.4564550+00:00


Description : Update portfolio group
At Time : 2019-02-18T13:28:10.6125300+00:00


Description : Delete portfolio from group
At Time : 2019-02-18T13:28:23.8992320+00:00


Description : Delete portfolio from group
At Time : 2019-02-18T13:28:24.2798240+00:00


Description : D

We can now verify that the group has indeed been deleted from our scope by getting all groups in each of our scopes, the second of which should be empty now.

In [156]:
# List groups in scope - show one scope with no groups
scope1_groups = client.list_portfolio_groups(scope = scope1)
scope2_groups = client.list_portfolio_groups(scope = scope2)
print("Groups in scope 1 : ")
for group in scope1_groups.values:
    print('    Scope : ', group.id.scope+', Code : ', group.id.code)

print("Groups in scope 2 : ")
for group in scope2_groups.values: # This one should be empty and not output anything
    print('    Scope : ', group.id.scope+', Code : ', group.id.code)

Groups in scope 1 : 
    Scope :  portfolio_demo_main, Code :  portfolio_group-a737
Groups in scope 2 : 


In [149]:
# # In case any extra groups are hanging around, delete them here
# # Find not matching group ids in each scope
# # # # # --- This might actually delete all groups??? --- # # # # # 
# scope1_group_ids_to_delete = []
# scope2_group_ids_to_delete = []

# for group in scope1_groups.values:
#     if group.id.code is not portfolio_group1.id.code :
#         scope1_group_ids_to_delete.append(group.id)

# for group in scope2_groups.values:
#     if group.id.code is not portfolio_group2.id.code :
#         scope2_group_ids_to_delete.append(group.id)
        
# for group in scope1_group_ids_to_delete:
#     client.delete_portfolio_group(
#         scope = group.scope, 
#         code = group.code)
    
# for group in scope2_group_ids_to_delete:
#     client.delete_portfolio_group(
#         scope = group.scope, 
#         code = group.code)    

If we list all the portfolios in our scopes we will see that they are all still there. There may be extra portfolios not created in this example, if this is the case, we can delete all these extra portfolios and keep only those we are interested in.

In [150]:

scope1_portfolios_list_response = client.list_portfolios_for_scope(
    scope = scope1,
    effective_at=None, 
    as_at=None, 
    sort_by=None, 
    start=None, 
    limit=None, 
    filter=None, 
    custom_headers=None, 
    raw=False)
scope2_portfolios_list_response = client.list_portfolios_for_scope(
    scope = scope2,
    effective_at=None, 
    as_at=None, 
    sort_by=None, 
    start=None, 
    limit=None, 
    filter=None, 
    custom_headers=None, 
    raw=False)

print('Portfolios remaining in scope ', scope1, ' :')
for portfolio in scope1_portfolios_list_response.values:
    prettyprint.portfolio_response(portfolio)
    print('\n')
    
print('Portfolios remaining in scope ', scope2, ' :')
for portfolio in scope2_portfolios_list_response.values:
    prettyprint.portfolio_response(portfolio)
    print('\n')

Portfolios remaining in scope  portfolio_demo_main  :
Portfolio Created
Scope: portfolio_demo_main
Code: main_105e4
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:50.002992+00:00



Portfolio Created
Scope: portfolio_demo_main
Code: main_58ff3
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:42.449959+00:00



Derived Portfolio Created
Scope: portfolio_demo_main
Code: derived_id-aec4
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:59.804735+00:00

Parent Portfolio Details
Code: main_105e4


Portfolio Created
Scope: portfolio_demo_main
Code: main_1c4ec
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-15 16:30:27.155121+00:00



Portfolio Created
Scope: portfolio_demo_main
Code: main_5bfa6
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-15 16:23:28.747308+00:00



Portfolio Created
Scope: portfo

The cells below allow us to delete the extra portfolios in the scope. Keep in mind, derived portfolios must be deleted first, as if a parent portfolio still contains a child, the deletion will fail.

In [157]:
# We can delete any unwanted portfolios now, lets say we only wish to keep the first 2 portfolios from our 
# original csv file, and the derived portfolio attached to the first portfolio. 
def extractIds(portfolioList):
    parent_ids = []
    derived_ids = []
    for folio in portfolioList:
        if folio.is_derived :
            derived_ids.append((folio.id.scope, folio.id.code))
        else :
            parent_ids.append((folio.id.scope, folio.id.code))
    return (parent_ids, derived_ids)
    
def returnNotMatches(a, b):
    return [x for x in a if x not in b]

# first combine all the portfolios in our two scopes
all_portfolios_in_scopes = extractIds(scope1_portfolios_list_response.values 
                                      + scope2_portfolios_list_response.values)
parent_portfolios = all_portfolios_in_scopes[0]
derived_portfolios = all_portfolios_in_scopes[1]

# then combine the portfolios we're interested in keeping into a list
portfolios_to_keep = extractIds([scope1_portfolios_list_response.values[0], 
                                 scope1_portfolios_list_response.values[1],
                                 derived_portfolio])
parents_to_keep = portfolios_to_keep[0]
derived_to_keep = portfolios_to_keep[1]

# we can now exclude portfolios to keep from the full list
parents_to_delete = returnNotMatches(parent_portfolios, parents_to_keep)
derived_to_delete = returnNotMatches(derived_portfolios, derived_to_keep)


In [158]:
# Next we delete the derived portfolios first
print("Deleting Derived Portfolios ... ")
for portfolio in derived_to_delete:
    response = client.delete_portfolio(
        scope = portfolio[0], 
        code = portfolio[1],
        effective_at = None, 
        custom_headers = None, 
        raw = False)
    print ('Portfolio deleted : ', portfolio[0], portfolio[1])
    print ('At time : ', str(response.as_at))
print('\n')
print("Deleting remaining Portfolios ... ")
# Then we proceed with deleting the remaining parent portfolios
for portfolio in parents_to_delete:
    response = client.delete_portfolio(
        scope = portfolio[0], 
        code = portfolio[1],
        effective_at = None, 
        custom_headers = None, 
        raw = False)
    print ('Portfolio deleted : ', portfolio[0], portfolio[1])
    print ('At time : ', str(response.as_at))

Deleting Derived Portfolios ... 
Portfolio deleted :  portfolio_demo_main derived_id-d952
At time :  2019-02-18 13:32:57.844628+00:00


Deleting remaining Portfolios ... 
Portfolio deleted :  portfolio_demo_main main_1c4ec
At time :  2019-02-18 13:32:58.278003+00:00
Portfolio deleted :  portfolio_demo_main main_5bfa6
At time :  2019-02-18 13:32:58.645951+00:00
Portfolio deleted :  portfolio_demo_main main_420cd
At time :  2019-02-18 13:32:59.018542+00:00
Portfolio deleted :  portfolio_demo_main main_3ea4e
At time :  2019-02-18 13:32:59.372617+00:00
Portfolio deleted :  portfolio_demo_main main_225a2
At time :  2019-02-18 13:32:59.734886+00:00
Portfolio deleted :  portfolio_demo_main main_6bc81
At time :  2019-02-18 13:33:00.484411+00:00
Portfolio deleted :  portfolio_demo_second second_2c207
At time :  2019-02-18 13:33:00.831934+00:00
Portfolio deleted :  portfolio_demo_second second_1884c
At time :  2019-02-18 13:33:01.192064+00:00


If we now re-list the portfolios in our scopes, we will see that only our two original portfolios remain, and one derived portfolio.

In [159]:
# We can re-run the list portfolios command to check what portfolios remain in each scope
scope1_portfolios_list_response = client.list_portfolios_for_scope(
    scope = scope1, 
    effective_at=None,
    as_at=None, 
    sort_by=None,
    start=None,
    limit=None,
    filter=None,
    custom_headers=None,
    raw=False)

scope2_portfolios_list_response = client.list_portfolios_for_scope(
    scope = scope2,
    effective_at=None,
    as_at=None, 
    sort_by=None,
    start=None,
    limit=None,
    filter=None,
    custom_headers=None,
    raw=False)

print('\033[91m','Portfolios remaining in scope: ','\033[0m', scope1, ':')
for portfolio in scope1_portfolios_list_response.values:
    prettyprint.portfolio_response(portfolio)
    print('\n')
    
print('\033[91m','Portfolios remaining in scope: ','\033[0m', scope2,':')
for portfolio in scope2_portfolios_list_response.values:
    prettyprint.portfolio_response(portfolio)
    print('\n')

 Portfolios remaining in scope:   portfolio_demo_main :
Portfolio Created
Scope: portfolio_demo_main
Code: main_105e4
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:50.002992+00:00



Portfolio Created
Scope: portfolio_demo_main
Code: main_58ff3
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:42.449959+00:00



Derived Portfolio Created
Scope: portfolio_demo_main
Code: derived_id-aec4
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-02-18 13:27:59.804735+00:00

Parent Portfolio Details
Code: main_105e4


 Portfolios remaining in scope:   portfolio_demo_second :
